## 1. Tianruan data check

In [ ]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle



In [36]:
pd.set_option('max_columns', 200)

In [1]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle

F2 = open('F:\\SZ.pkl', 'rb')
SZ = pickle.load(F2)
p1 = SZ[SZ["date"].isin(['2019-01-09', "2019-03-14", "2019-03-15"])].groupby(["ID", "date"]).first().reset_index()
p2 = SZ[SZ["date"].isin(['2019-01-09', "2019-03-14", "2019-03-15"])].groupby(["ID", "date"]).last().reset_index()
p3 = SZ[~SZ["date"].isin(['2019-01-09', "2019-03-14", "2019-03-15"])]
SZ = pd.concat([p1, p3])

readPath = r'\\192.168.10.30\Kevin_zhenyu\day_stock_20200416\***'
dataPathLs = np.array(glob.glob(readPath))
dataPathLs = dataPathLs[[np.array([os.path.basename(i).split('.')[0][:2] == 'SZ' for i in dataPathLs])]]
db = pd.DataFrame()
for p in dataPathLs:
    dayData = pd.read_csv(p, compression='gzip')
    db = pd.concat([db, dayData])
date = SZ["date"].unique()
date1 = db["date"].unique()
date1 = date1[(date1>=date.min()) & (date1 <= date.max())]
display(set(date1) - set(date))
db = db[db["date"].isin(date)]
s1 = SZ["ID"].unique()
s2 = db["ID"].unique()
ss = list(set(s1) & set(s2))
SZ = SZ[SZ["ID"].isin(ss)]
db = db[db["ID"].isin(ss)]

# 注意
# 1. diff里包括所有的不匹配，可能是close price造成的，需要在下一步验证
# 2. 第二次merge仍有不匹配，继续查找原因
# 3. 天软数据包括当天停牌的股票，openPrice == 0
# 4. 实际的收盘价调整是两次merge相减的结果
# 5. 深交所数据那天会重复
# 6. SZ包括当天停牌的股票，SH应该不包括，再确认
df = pd.merge(SZ, db, on=["ID", "date", "d_open", "d_yclose","d_high", "d_low", "d_close", "d_volume"], how="outer")
if sum(df["d_amount_x"].isnull()) != 0:
    diff = df[df["d_amount_x"].isnull()][["ID", "date"]]
    df = pd.merge(SZ, db, on=["ID", "date", "d_open", "d_yclose","d_high", "d_low", "d_volume"], how="outer")
    if sum(df["d_amount_x"].isnull()) == 0:
        print("Not all stocks have same close price as Tianruan")
    else:
        print(df[df["d_amount_x"].isnull()])
        print(df[(df["d_amount_y"].isnull()) & (df["d_open"] != 0)])
else:
    print("All equal")
print(diff)
display(diff["ID"].unique())
display(len(diff["ID"].unique()))
display(diff["date"].unique())
display(len(diff["date"].unique()))

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


{'2019-04-01',
 '2019-04-04',
 '2019-05-16',
 '2019-08-22',
 '2019-09-03',
 '2019-11-19',
 '2019-11-20'}

Not all stocks have same close price as Tianruan
              ID        date
474936  SZ000411  2019-06-13
474937  SZ000411  2019-08-12
474938  SZ300636  2019-01-22
474939  SZ300636  2019-01-23
474940  SZ002853  2019-05-14
...          ...         ...
475155  SZ000899  2019-06-10
475156  SZ002315  2019-01-22
475157  SZ002620  2019-06-25
475158  SZ300218  2019-01-10
475159  SZ002715  2019-07-30

[224 rows x 2 columns]


array(['SZ000411', 'SZ300636', 'SZ002853', 'SZ300330', 'SZ000010',
       'SZ002883', 'SZ002029', 'SZ002826', 'SZ002890', 'SZ300619',
       'SZ300486', 'SZ002802', 'SZ002778', 'SZ000421', 'SZ000809',
       'SZ002107', 'SZ300668', 'SZ002150', 'SZ002336', 'SZ002749',
       'SZ300419', 'SZ000893', 'SZ300421', 'SZ002096', 'SZ002053',
       'SZ002791', 'SZ000782', 'SZ300243', 'SZ000153', 'SZ002781',
       'SZ002633', 'SZ300439', 'SZ002678', 'SZ300677', 'SZ300354',
       'SZ002868', 'SZ300305', 'SZ300151', 'SZ300658', 'SZ000548',
       'SZ002339', 'SZ300483', 'SZ002062', 'SZ002652', 'SZ002775',
       'SZ002735', 'SZ300434', 'SZ300530', 'SZ002187', 'SZ002598',
       'SZ000978', 'SZ002290', 'SZ300641', 'SZ300695', 'SZ300250',
       'SZ002671', 'SZ002833', 'SZ002520', 'SZ300617', 'SZ300669',
       'SZ002561', 'SZ000532', 'SZ300334', 'SZ002394', 'SZ002738',
       'SZ000902', 'SZ000403', 'SZ002559', 'SZ002337', 'SZ300065',
       'SZ002286', 'SZ000428', 'SZ300729', 'SZ002469', 'SZ0026

146

array(['2019-06-13', '2019-08-12', '2019-01-22', '2019-01-23',
       '2019-05-14', '2019-02-11', '2019-08-14', '2019-09-04',
       '2019-11-18', '2019-07-17', '2019-08-13', '2019-06-04',
       '2019-07-26', '2019-05-17', '2019-10-23', '2019-11-27',
       '2019-08-28', '2019-01-21', '2019-05-24', '2019-11-13',
       '2019-06-18', '2019-08-01', '2019-08-30', '2019-11-07',
       '2019-09-10', '2019-12-02', '2019-12-06', '2019-05-31',
       '2019-12-03', '2019-11-26', '2019-07-09', '2019-11-21',
       '2019-08-09', '2019-01-09', '2019-02-12', '2019-05-22',
       '2019-07-25', '2019-08-06', '2019-06-14', '2019-09-17',
       '2019-01-30', '2019-07-24', '2019-08-02', '2019-01-03',
       '2019-01-10', '2019-01-08', '2019-09-26', '2019-10-17',
       '2019-06-05', '2019-08-21', '2019-08-26', '2019-12-04',
       '2019-07-29', '2019-04-18', '2019-11-29', '2019-07-16',
       '2019-10-08', '2019-11-01', '2019-10-15', '2019-08-07',
       '2019-07-10', '2019-06-10', '2019-08-27', '2019-

118

In [3]:
display(db[(db["ID"] == "SZ000411") & (db["date"] == "2019-06-13")])
display(SZ[(SZ["ID"] == "SZ000411") & (SZ["date"] == "2019-06-13")])
display(SZ[(SZ["ID"] == "SZ000411") & (SZ["date"] == "2019-06-14")])

,ID,date,d_open,d_high,d_low,d_close,d_yclose,d_cumprodCAA,d_dayReturn,d_5dayReturn,...,allZT,hasZT,isZT,allDT,hasDT,isDT,tmrHalted,haltedDays,marketShares,totalShares
1645,SZ000411,2019-06-13,12.28,12.53,12.24,12.41,12.27,0.989683,0.01141,0.004045,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,207310071.0,207449946.0


,ID,date,d_open,d_yclose,d_high,d_low,d_close,d_volume,d_amount
94376,SZ000411,2019-06-13,12.28,12.27,12.53,12.24,12.4,879064.0,10889547.12


,ID,date,d_open,d_yclose,d_high,d_low,d_close,d_volume,d_amount
96573,SZ000411,2019-06-14,12.46,12.41,12.52,12.18,12.19,748231.0,9231817.64


In [1]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle

F1 = open('F:\\SH1.pkl', 'rb')
SH = pickle.load(F1)

readPath = r'\\192.168.10.30\Kevin_zhenyu\day_stock_20200416\***'
dataPathLs = np.array(glob.glob(readPath))
dataPathLs = dataPathLs[[np.array([os.path.basename(i).split('.')[0][:2] == 'SH' for i in dataPathLs])]]
db = pd.DataFrame()
for p in dataPathLs:
    dayData = pd.read_csv(p, compression='gzip')
    db = pd.concat([db, dayData])
date = SH["date"].unique()
date1 = db["date"].unique()
date1 = date1[(date1>=date.min()) & (date1 <= date.max())]
display(set(date1) - set(date))
db = db[db["date"].isin(date)]
s1 = SH["ID"].unique()
s2 = db["ID"].unique()
ss = list(set(s1) & set(s2))
SH = SH[SH["ID"].isin(ss)]
db = db[db["ID"].isin(ss)]

# 注意
# 1. diff里包括所有的不匹配，可能是close price造成的，需要在下一步验证
# 2. 第二次merge仍有不匹配，继续查找原因
# 3. 天软数据包括当天停牌的股票，openPrice == 0
# 4. 实际的收盘价调整是两次merge相减的结果
# 5. 深交所数据那天会重复
# 6. SZ包括当天停牌的股票，SH应该不包括，再确认
df = pd.merge(SH, db, on=["ID", "date", "d_open", "d_yclose","d_high", "d_low", "d_close", "d_volume"], how="outer")
if sum(df["d_amount_x"].isnull()) != 0:
    diff = df[df["d_amount_x"].isnull()][["ID", "date"]]
    df = pd.merge(SH, db, on=["ID", "date", "d_open", "d_yclose","d_high", "d_low", "d_volume"], how="outer")
    if sum(df["d_amount_x"].isnull()) == 0:
        print("Not all stocks have same close price as Tianruan")
    else:
        print(df[df["d_amount_x"].isnull()])
        print(df[(df["d_amount_y"].isnull()) & (df["d_open"] != 0)])
else:
    print("All equal")
print(diff)
display(diff["ID"].unique())
display(len(diff["ID"].unique()))
display(diff["date"].unique())
display(len(diff["date"].unique()))

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]


set()

Not all stocks have same close price as Tianruan
             ID        date
79658  SH603990  2019-01-14
79659  SH603990  2019-03-21
79660  SH603203  2019-01-30
79661  SH600819  2019-01-17
79662  SH600106  2019-01-21
...         ...         ...
79720  SH603991  2019-01-28
79721  SH600539  2019-01-09
79722  SH600539  2019-01-10
79723  SH600539  2019-01-15
79724  SH603269  2019-01-29

[67 rows x 2 columns]


## 2. SH data check

In [1]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle
from decimal import Decimal, ROUND_HALF_UP
import datetime

startDate = "20190201"
endDate = "20191210"

readPath = 'F:\\SH\\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i) for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]

startTm = datetime.datetime.now()
diff = {}
for col in ['Date', 'Data', 'Condition', 'Prob', 'Time']:
    diff[col] = []

for p in dataPathLs:
    print("*************************************************************************************************")
    F1 = open(p, 'rb')
    snapshot = pickle.load(F1)
    print(datetime.datetime.now() - startTm)
    
    snapshot["DateTime"] = snapshot["DateTime"].apply(lambda x: int(x))
    dd = int(snapshot["DateTime"].iloc[0]//1000000 * 1000000)
    snapshot['time'] = (snapshot["DateTime"] - dd)*1000
    
    path = "J:\\recordData\\logs_" + os.path.basename(p).split('.')[0] + "_zs_92_01_day_data\\mdLog_SH_***" 
    if len(np.array(glob.glob(path))) == 0:
        print(os.path.basename(p).split('.')[0])
        print("92 SSE snapshot data is missing")
        continue
    if len(np.array(glob.glob(path))) == 2:
        logSH1 = pd.read_csv(np.array(glob.glob(path))[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
        logSH2 = pd.read_csv(np.array(glob.glob(path))[1], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
        logSH = pd.concat([logSH1, logSH2])
    else:
        logSH = pd.read_csv(np.array(glob.glob(path))[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
    print(datetime.datetime.now() - startTm)
    
    logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    logSH1 = snapshot[["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice",'TotalBidQty', 'TotalOfferQty','time']]
    
    # SH lv2 data check here
    display(os.path.basename(p).split('.')[0] + " SH lv2 check:")
    in_dex = logSH[logSH["source"] == 5]["StockID"].unique()
    data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH["time"] > 92500000) & (logSH["time"] <= 145700000) & (logSH["cum_volume"] > 0)
              & (logSH["source"] == 4)]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1["time"] > 92500000) & (logSH1["time"] <= 145700000) & (logSH1["cum_volume"] > 0)]
    data1["openPrice"] = data1.groupby("StockID")["openPrice"].transform("max")
    data2["openPrice"] = data2.groupby("StockID")["openPrice"].transform("max")
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    del data1
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    del data2
    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    if n1 > n2:
        data1_1 = data1_1[data1_1["StockID"].isin(data2_1["StockID"].unique())]
    if n2 > n1:
        data2_1 = data2_1[data2_1["StockID"].isin(data1_1["StockID"].unique())]
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["time_x"].count()
    n2 = test["time_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    diff["Date"].append(os.path.basename(p).split('.')[0])
    diff["Data"].append("lv2 data")
    re = test[(~np.isnan(test["time_y"]))&(~np.isnan(test["time_x"]))]
    diff["Time"].append(re[re["time_x"] == re["time_y"]].shape[0]/re.shape[0])
    if (n2 < len1) & (n1 < len1):
        display("SSE lv2 test is not complete:")
        display(test[np.isnan(test["time_y"])])
        print(len(test[np.isnan(test["time_y"])])/n1)
        if len(test[np.isnan(test["time_y"])])/n1 > 0.001:
            diff["Condition"].append("SSE lv2 test is not complete")
            diff["Prob"].append(len(test[np.isnan(test["time_y"])])/n1)
        else:
            diff["Condition"].append("SSE lv2 test is complete")
            diff["Prob"].append(len(test[np.isnan(test["time_y"])])/n1)
    
    if (n2 < len1) & (n1 == len1):
        display("SSE lv2 test is not complete:")
        display(test[np.isnan(test["time_y"])])
        print(len(test[np.isnan(test["time_y"])]))
        if len(test[np.isnan(test["time_y"])]) > 10:
            diff["Condition"].append("SSE lv2 test is not complete")
            diff["Prob"].append(len(test[np.isnan(test["time_y"])]))
        else:
            diff["Condition"].append("SSE lv2 test is complete")
            diff["Prob"].append(len(test[np.isnan(test["time_y"])]))
          
    if len1 == n2:
        display("SSE lv2 test is complete")
        diff["Condition"].append("SSE lv2 test is complete")
        diff["Prob"].append('C')
    
    # SH snapshot data check here
    display(os.path.basename(p).split('.')[0] + " SH index check:")
    data1 = logSH[logSH["StockID"].isin(in_dex) & (logSH["time"] > 92500000) & (logSH["time"] <= 145700000)]
    data2 = logSH1[logSH1["StockID"].isin(in_dex) & (logSH1["time"] > 92500000) & (logSH1["time"] <= 145700000)]
    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
    data1["openPrice"] = data1.groupby("StockID")["openPrice"].transform("max")
    data2["openPrice"] = data2.groupby("StockID")["openPrice"].transform("max")
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    columns = ["StockID", "cum_volume", "close", "openPrice"]
    data1_1["openPrice"] = data1_1["openPrice"].apply(lambda x: float(Decimal(str(x)).quantize(Decimal('0.000'), rounding=ROUND_HALF_UP)))
    data1_1["close"] = data1_1["close"].apply(lambda x: float(Decimal(str(x)).quantize(Decimal('0.000'), rounding=ROUND_HALF_UP)))
    data2_1["openPrice"] = data2_1["openPrice"].apply(lambda x: float(Decimal(str(x)).quantize(Decimal('0.000'), rounding=ROUND_HALF_UP)))
    data2_1["close"] = data2_1["close"].apply(lambda x: float(Decimal(str(x)).quantize(Decimal('0.000'), rounding=ROUND_HALF_UP)))
    data1_1["openPrice"] = data1_1["openPrice"].apply(lambda x: float(Decimal(str(x)).quantize(Decimal('0.00'), rounding=ROUND_HALF_UP)))
    data1_1["close"] = data1_1["close"].apply(lambda x: float(Decimal(str(x)).quantize(Decimal('0.00'), rounding=ROUND_HALF_UP)))
    data2_1["openPrice"] = data2_1["openPrice"].apply(lambda x: float(Decimal(str(x)).quantize(Decimal('0.00'), rounding=ROUND_HALF_UP)))
    data2_1["close"] = data2_1["close"].apply(lambda x: float(Decimal(str(x)).quantize(Decimal('0.00'), rounding=ROUND_HALF_UP)))
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["time_x"].count()
    n2 = test["time_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    diff["Date"].append(os.path.basename(p).split('.')[0])
    diff["Data"].append("index data")
    re = test[(~np.isnan(test["time_y"]))&(~np.isnan(test["time_x"]))]
    diff["Time"].append(re[re["time_x"] == re["time_y"]].shape[0]/re.shape[0])
    # Here!!! index double check wether n1 == n2
    if n2 < len1:
        display("SSE index test is not complete:")
        display(test[np.isnan(test["time_y"])])
        piu = pd.merge(pd.merge(data1_1, test[np.isnan(test["time_y"])][["StockID", "cum_volume"]]), pd.merge(data2_1, test[np.isnan(test["time_y"])][["StockID", "cum_volume"]]),
         on=["StockID", "cum_volume"])
        print(len(test[np.isnan(test["time_y"])]))
        if all(abs(piu["close_x"] - piu["close_y"]) <= 0.011) & all(abs(piu["openPrice_x"] - piu["openPrice_y"]) <= 0.011):
            display("test is complete, the difference is caused by round difference")
            diff["Condition"].append("SSE index test is complete with round difference")
            diff["Prob"].append('C')
        else:
            diff["Condition"].append("SSE index test is not complete")
            diff["Prob"].append(len(test[np.isnan(test["time_y"])]))
    if len1 == n2:
        display("SSE index test is complete")
        diff["Condition"].append("SSE index test is complete")
        diff["Prob"].append('C')

   
    
#     # SH trade data check here
#     path = "G:\\recordData\\logs_" + os.path.basename(p).split('.')[0] + "_zs_92_01_day_data\\mdTradeLog_***" 
#     TradeLog = pd.read_csv(np.array(glob.glob(path))[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
#                                                  "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
#                                                  "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
#                                                  "OfferApplSeqNum"]]
#     SH = TradeLog[TradeLog["exchId"] == 1]
#     del TradeLog
#     colnames=['SecurityID', 'TradeTime', 'TradePrice', 'TradeQty','TradeAmount','BuyNo','SellNo'] 
#     tick = pd.read_csv(path2[i], names=colnames, header=None)
#     tick = tick.rename(columns={"TradeAmount":"TradeMoney", "BuyNo":"BidApplSeqNum","SellNo":"OfferApplSeqNum"})
#     tick = tick.dropna(axis=0)
#     tick["TradeTime"] = tick["TradeTime"].apply(lambda x: int(x))
#     tick["TransactTime"] = (tick['TradeTime'] - 2019120900000000)*10
#     tick["TradePrice"] = tick["TradePrice"]*10000
#     tick["TradeMoney"] = tick["TradeMoney"]*10000
#     display(os.path.basename(p) + " SH trade check:")
#     tick["TradeMoney"] = tick["TradeMoney"].round(2)
#     tick["TradePrice"] = tick["TradePrice"].round(2)
#     columns = ["TransactTime", "SecurityID", "TradeQty","TradePrice","TradeMoney",
#            "BidApplSeqNum", "OfferApplSeqNum"]
#     n1 = len(SH["SecurityID"].unique())
#     n2 = len(tick["SecurityID"].unique())
#     if n1 > n2:
#         SH = SH[SH["SecurityID"].isin(tick["SecurityID"].unique())]
#     if n2 > n1:
#         tick = tick[tick["SecurityID"].isin(SH["SecurityID"].unique())]
#     re = pd.merge(SH, tick, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
#     n1 = re["sequenceNo"].count()
#     n2 = re["TradeTime"].count()
#     len1 = len(re)
#     print(n1)
#     print(n2)
#     print(len1)
#     print("-----------------------------------------------")
#     if n2 < len1:
#         display("SSE trade test is not complete:")
#         display(re[np.isnan(re["TradeTime"])])
#         print(len(re[np.isnan(re["TradeTime"])]))
#         print(np.sort(re[np.isnan(re["TradeTime"])]["TransactTime"].unique()))
#         print(len(re[np.isnan(re["TradeTime"])]["SecurityID"].unique()))
#         print(re[np.isnan(re["TradeTime"])]["SecurityID"].unique())
#     if (len1 == n2) & (n1 < len1):
#         display("SSE trade baseline is not complete:")
#         display(re[np.isnan(re["sequenceNo"])])
#         print(np.sort(re[np.isnan(re["sequenceNo"])]["TransactTime"].unique()))
#         print(len(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique()))
#         print(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique())
#         display(n2-n1)
#     if (len1 == n2) & (len1 == n1):
#         display("SSE trade baseline and test exactly same")
    
    
diff = pd.DataFrame(diff)
diff.to_csv("F:\\SH_check_result2.csv")
print(diff)

*************************************************************************************************
0:00:38.079676
0:03:06.153334


'20190201 SH lv2 check:'

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


3945769
3945769
3945769
-----------------------------------------------


'SSE lv2 test is complete'

...

'20190426 SH index check:'

11392
11392
11392
-----------------------------------------------


'SSE index test is complete'

*************************************************************************************************


MemoryError: 

In [1]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle
from decimal import Decimal, ROUND_HALF_UP
import datetime

startDate = "20190427"
endDate = "20191210"

readPath = 'F:\\SH\\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i) for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]

startTm = datetime.datetime.now()
diff = {}
for col in ['Date', 'Data', 'Condition', 'Prob', 'Time']:
    diff[col] = []

for p in dataPathLs:
    print("*************************************************************************************************")
    F1 = open(p, 'rb')
    snapshot = pickle.load(F1)
    print(datetime.datetime.now() - startTm)
    
    snapshot["DateTime"] = snapshot["DateTime"].apply(lambda x: int(x))
    dd = int(snapshot["DateTime"].iloc[0]//1000000 * 1000000)
    snapshot['time'] = (snapshot["DateTime"] - dd)*1000
    
    path = "J:\\recordData\\logs_" + os.path.basename(p).split('.')[0] + "_zs_92_01_day_data\\mdLog_SH_***" 
    if len(np.array(glob.glob(path))) == 0:
        print(os.path.basename(p).split('.')[0])
        print("92 SSE snapshot data is missing")
        continue
    if len(np.array(glob.glob(path))) == 2:
        logSH1 = pd.read_csv(np.array(glob.glob(path))[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
        logSH2 = pd.read_csv(np.array(glob.glob(path))[1], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
        logSH = pd.concat([logSH1, logSH2])
    else:
        logSH = pd.read_csv(np.array(glob.glob(path))[0], encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
    print(datetime.datetime.now() - startTm)
    
    logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    logSH1 = snapshot[["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice",'TotalBidQty', 'TotalOfferQty','time']]
    
    # SH lv2 data check here
    display(os.path.basename(p).split('.')[0] + " SH lv2 check:")
    in_dex = logSH[logSH["source"] == 5]["StockID"].unique()
    data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH["time"] > 92500000) & (logSH["time"] <= 145700000) & (logSH["cum_volume"] > 0)
              & (logSH["source"] == 4)]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1["time"] > 92500000) & (logSH1["time"] <= 145700000) & (logSH1["cum_volume"] > 0)]
    data1["openPrice"] = data1.groupby("StockID")["openPrice"].transform("max")
    data2["openPrice"] = data2.groupby("StockID")["openPrice"].transform("max")
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    del data1
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    del data2
    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    if n1 > n2:
        data1_1 = data1_1[data1_1["StockID"].isin(data2_1["StockID"].unique())]
    if n2 > n1:
        data2_1 = data2_1[data2_1["StockID"].isin(data1_1["StockID"].unique())]
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["time_x"].count()
    n2 = test["time_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    diff["Date"].append(os.path.basename(p).split('.')[0])
    diff["Data"].append("lv2 data")
    re = test[(~np.isnan(test["time_y"]))&(~np.isnan(test["time_x"]))]
    diff["Time"].append(re[re["time_x"] == re["time_y"]].shape[0]/re.shape[0])
    if (n2 < len1) & (n1 < len1):
        display("SSE lv2 test is not complete:")
        display(test[np.isnan(test["time_y"])])
        print(len(test[np.isnan(test["time_y"])])/n1)
        if len(test[np.isnan(test["time_y"])])/n1 > 0.001:
            diff["Condition"].append("SSE lv2 test is not complete")
            diff["Prob"].append(len(test[np.isnan(test["time_y"])])/n1)
        else:
            diff["Condition"].append("SSE lv2 test is complete")
            diff["Prob"].append(len(test[np.isnan(test["time_y"])])/n1)
    
    if (n2 < len1) & (n1 == len1):
        display("SSE lv2 test is not complete:")
        display(test[np.isnan(test["time_y"])])
        print(len(test[np.isnan(test["time_y"])]))
        if len(test[np.isnan(test["time_y"])]) > 10:
            diff["Condition"].append("SSE lv2 test is not complete")
            diff["Prob"].append(len(test[np.isnan(test["time_y"])]))
        else:
            diff["Condition"].append("SSE lv2 test is complete")
            diff["Prob"].append(len(test[np.isnan(test["time_y"])]))
          
    if len1 == n2:
        display("SSE lv2 test is complete")
        diff["Condition"].append("SSE lv2 test is complete")
        diff["Prob"].append('C')
    
    # SH snapshot data check here
    display(os.path.basename(p).split('.')[0] + " SH index check:")
    data1 = logSH[logSH["StockID"].isin(in_dex) & (logSH["time"] > 92500000) & (logSH["time"] <= 145700000)]
    data2 = logSH1[logSH1["StockID"].isin(in_dex) & (logSH1["time"] > 92500000) & (logSH1["time"] <= 145700000)]
    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
    data1["openPrice"] = data1.groupby("StockID")["openPrice"].transform("max")
    data2["openPrice"] = data2.groupby("StockID")["openPrice"].transform("max")
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    columns = ["StockID", "cum_volume", "close", "openPrice"]
    data1_1["openPrice"] = data1_1["openPrice"].apply(lambda x: float(Decimal(str(x)).quantize(Decimal('0.000'), rounding=ROUND_HALF_UP)))
    data1_1["close"] = data1_1["close"].apply(lambda x: float(Decimal(str(x)).quantize(Decimal('0.000'), rounding=ROUND_HALF_UP)))
    data2_1["openPrice"] = data2_1["openPrice"].apply(lambda x: float(Decimal(str(x)).quantize(Decimal('0.000'), rounding=ROUND_HALF_UP)))
    data2_1["close"] = data2_1["close"].apply(lambda x: float(Decimal(str(x)).quantize(Decimal('0.000'), rounding=ROUND_HALF_UP)))
    data1_1["openPrice"] = data1_1["openPrice"].apply(lambda x: float(Decimal(str(x)).quantize(Decimal('0.00'), rounding=ROUND_HALF_UP)))
    data1_1["close"] = data1_1["close"].apply(lambda x: float(Decimal(str(x)).quantize(Decimal('0.00'), rounding=ROUND_HALF_UP)))
    data2_1["openPrice"] = data2_1["openPrice"].apply(lambda x: float(Decimal(str(x)).quantize(Decimal('0.00'), rounding=ROUND_HALF_UP)))
    data2_1["close"] = data2_1["close"].apply(lambda x: float(Decimal(str(x)).quantize(Decimal('0.00'), rounding=ROUND_HALF_UP)))
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["time_x"].count()
    n2 = test["time_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    diff["Date"].append(os.path.basename(p).split('.')[0])
    diff["Data"].append("index data")
    re = test[(~np.isnan(test["time_y"]))&(~np.isnan(test["time_x"]))]
    diff["Time"].append(re[re["time_x"] == re["time_y"]].shape[0]/re.shape[0])
    if n2 < len1:
        display("SSE index test is not complete:")
        display(test[np.isnan(test["time_y"])])
        piu = pd.merge(pd.merge(data1_1, test[np.isnan(test["time_y"])][["StockID", "cum_volume"]]), pd.merge(data2_1, test[np.isnan(test["time_y"])][["StockID", "cum_volume"]]),
         on=["StockID", "cum_volume"])
        print(len(test[np.isnan(test["time_y"])]))
        if all(abs(piu["close_x"] - piu["close_y"]) <= 0.011) & all(abs(piu["openPrice_x"] - piu["openPrice_y"]) <= 0.011):
            display("test is complete, the difference is caused by round difference")
            diff["Condition"].append("SSE index test is complete with round difference")
            diff["Prob"].append('C')
        else:
            diff["Condition"].append("SSE index test is not complete")
            diff["Prob"].append(len(test[np.isnan(test["time_y"])]))
    if len1 == n2:
        display("SSE index test is complete")
        diff["Condition"].append("SSE index test is complete")
        diff["Prob"].append('C')
       
    
diff = pd.DataFrame(diff)
diff.to_csv("F:\\SH_check_result2.csv")
print(diff)

*************************************************************************************************
0:00:50.608045
0:04:04.928428


'20190429 SH lv2 check:'

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


5042984
5043119
5043673
-----------------------------------------------


'SSE lv2 test is not complete:'

...

'SSE index test is not complete:'

,index_x,clockAtArrival,sequenceNo,source,StockID,exchange,time_x,cum_volume,cum_amount_x,close,...,ask4p_y,ask5p_y,ask1q_y,ask2q_y,ask3q_y,ask4q_y,ask5q_y,TotalBidQty,TotalOfferQty,time_y
2627,2967385.0,1.575858e+15,32401625.0,5.0,905,SH,102432000.0,35776152,3.871161e+10,5032.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8759,8523842.0,1.575871e+15,70020406.0,5.0,905,SH,140207000.0,69634986,7.402243e+10,5017.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9823,9450866.0,1.575873e+15,76733264.0,5.0,905,SH,142417000.0,75344580,7.964790e+10,5020.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


3


'test is complete, the difference is caused by round difference'

         Date        Data                                         Condition  \
0    20190429    lv2 data                          SSE lv2 test is complete   
1    20190429  index data                        SSE index test is complete   
2    20190430    lv2 data                          SSE lv2 test is complete   
3    20190430  index data                        SSE index test is complete   
4    20190506    lv2 data                          SSE lv2 test is complete   
..        ...         ...                                               ...   
281  20191205  index data  SSE index test is complete with round difference   
282  20191206    lv2 data                      SSE lv2 test is not complete   
283  20191206  index data  SSE index test is complete with round difference   
284  20191209    lv2 data                      SSE lv2 test is not complete   
285  20191209  index data  SSE index test is complete with round difference   

            Prob      Time  
0    0.000109856  0.99